In [1]:
import bs4 as bs
import urllib.request
import pickle
import pandas as pd
import numpy as np
#https://fantasy.premierleague.com/statistics
sauce = urllib.request.urlopen('https://www.serebii.net/pokemon/nationalpokedex.shtml').read()

In [2]:
soup = bs.BeautifulSoup(sauce,'lxml')

# EXPERIMENT WITH WHERE THINGS ARE

In [3]:
print(soup.title)
print(soup.title.name)
print(soup.title.string)
print(soup.title.text)

<title>Pokémon - National Pokédex Pokémon</title>
title
Pokémon - National Pokédex Pokémon
Pokémon - National Pokédex Pokémon


In [4]:
#for paragraph in soup.find_all('td'):
    #print(paragraph.text)

In [5]:
table = soup.table
#print(table)

In [6]:
table_rows=table.find_all('tr')
len(table_rows)

1812

In [7]:
num = []
name = []
typ = []
ability = []
for tr in table_rows:
    td = tr.find_all('td')
    #temp_num = td[0].text
    #temp_name = td[2].text
    #temp_type = td[3].text
    #temp_abi = td[4].text
    num.append(td[0].text)
    #name.append(td[2].text)
    #typ.append(td[3].text)
    #ability.append(td[4].text)
df = pd.DataFrame({'ID':num})
#df['Name'] = name
#df['Type'] = typ
#df['Ability'] = ability
df = df.iloc[::2]
df = df.replace('\n','', regex=True)
df = df.replace('\t','', regex=True)
df = df.replace('\r','', regex=True)
df = df.replace('#','', regex=True)
df = df.iloc[1:]

In [8]:
df

,ID
2,001
4,002
6,003
8,004
10,005
...,...
1802,901
1804,902
1806,903
1808,904


# READ_HTML

In [9]:
dfs = pd.read_html("https://www.serebii.net/pokemon/gen5pokemon.shtml")
dfs

[       0    1         2     3                       4           5   \
 0     No.  Pic      Name  Type               Abilities  Base Stats   
 1     No.  Pic      Name  Type               Abilities          HP   
 2    #494  NaN   Victini   NaN            Victory Star         100   
 3    #495  NaN     Snivy   NaN       Overgrow Contrary          45   
 4    #496  NaN   Servine   NaN       Overgrow Contrary          60   
 ..    ...  ...       ...   ...                     ...         ...   
 153  #645  NaN  Landorus   NaN  Sand Force Sheer Force          89   
 154  #646  NaN    Kyurem   NaN                Pressure         125   
 155  #647  NaN    Keldeo   NaN               Justified          91   
 156  #648  NaN  Meloetta   NaN            Serene Grace         100   
 157  #649  NaN  Genesect   NaN                Download          71   
 
              6           7           8           9           10  
 0    Base Stats  Base Stats  Base Stats  Base Stats  Base Stats  
 1          

# MANUAL TABLE DATA

In [10]:
table_data=table.find_all('td')
#table_data = np.delete(table_data,5,0)
len(table_data)

10872

In [11]:
num = []
name = []
ability = []
hp = []
atk = []
spatk = []
defence = []
spdef = []
spd = []
typ = []
i = 0
for td in table_data:
    if i%12 == 11:
        spd.append(td.text)
    if i%12 == 0:
        num.append(td.text)
    if i%12 == 3:
        name.append(td.text)
    if i%12 == 4:
        a = td.find_all('a')
        typ.append(a)
    if i%12 == 5:
        a = td.find_all('a')
        ability.append(a)
    if i%12 == 6:
        hp.append(td.text)
    if i%12 == 7:
        atk.append(td.text)
    if i%12 == 8:
        defence.append(td.text)
    if i%12 == 9:
        spatk.append(td.text)
    if i%12 == 10:
        spdef.append(td.text)
    i = i+1

In [12]:
df = pd.DataFrame({'ID':num})
df['NAME'] = name
df['HP'] = hp
df['ATK'] = atk
df['DEF'] = defence
df['SPATK'] = spatk
df['SPDEF'] = spdef
df['SPD'] = spd
df['TYPE'] = typ
df['ABILITY'] = ability
df = df.replace('\n','', regex=True)
df = df.replace('\t','', regex=True)
df = df.replace('\r','', regex=True)
df = df.replace('#','', regex=True)
df = df.iloc[1:]
# seperate ability from one column to three
df[['TYPE1','TYPE2']] = pd.DataFrame(df.TYPE.tolist(), index= df.index)
df[['ABILITY1','ABILITY2','ABILITY3']] = pd.DataFrame(df.ABILITY.tolist(), index= df.index)
df = df.drop(['ABILITY'],axis=1)
df = df.drop(['TYPE'],axis=1)
df.head(5)

,ID,NAME,HP,ATK,DEF,SPATK,SPDEF,SPD,TYPE1,TYPE2,ABILITY1,ABILITY2,ABILITY3
1,001,Bulbasaur,45,49,49,65,65,45,[[]],[[]],[Overgrow],[Chlorophyll],None
2,002,Ivysaur,60,62,63,80,80,60,[[]],[[]],[Overgrow],[Chlorophyll],None
3,003,Venusaur,80,82,83,100,100,80,[[]],[[]],[Overgrow],[Chlorophyll],None
4,004,Charmander,39,52,43,60,50,65,[[]],None,[Blaze],[Solar Power],None
5,005,Charmeleon,58,64,58,80,65,80,[[]],None,[Blaze],[Solar Power],None


In [13]:
#convert columns to different types
df['HP']=df['HP'].values.astype(str).astype(int)
df['ATK']=df['ATK'].values.astype(str).astype(int)
df['DEF']=df['DEF'].values.astype(str).astype(int)
df['SPATK']=df['SPATK'].values.astype(str).astype(int)
df['SPDEF']=df['SPDEF'].values.astype(str).astype(int)
df['SPD']=df['SPD'].values.astype(str).astype(int)
df['NAME']=df['NAME'].values.astype(str)
df['ID']=df['ID'].values.astype(str).astype(int)
df['ABILITY1']=df['ABILITY1'].astype(str).replace('<[^>]*>','', regex=True) #remove html tags (assuming no characters inside the text)
df['ABILITY2']=df['ABILITY2'].astype(str).replace('<[^>]*>','', regex=True) #remove html tags (assuming no characters inside the text)
df['ABILITY3']=df['ABILITY3'].astype(str).replace('<[^>]*>','', regex=True) #remove html tags (assuming no characters inside the text)
df['TYPE1'] = df['TYPE1'].astype(str).replace('<a href="/pokemon/type/','',regex=True).replace('".*','',regex=True)
df['TYPE2'] = df['TYPE2'].astype(str).replace('<a href="/pokemon/type/','',regex=True).replace('".*','',regex=True)
df['TOTAL'] = df['HP']+df['ATK']+df['DEF']+df['SPATK']+df['SPDEF']+df['SPD']

In [14]:
df.dtypes

ID           int64
NAME        object
HP           int64
ATK          int64
DEF          int64
SPATK        int64
SPDEF        int64
SPD          int64
TYPE1       object
TYPE2       object
ABILITY1    object
ABILITY2    object
ABILITY3    object
TOTAL        int64
dtype: object

In [15]:
df

,ID,NAME,HP,ATK,DEF,SPATK,SPDEF,SPD,TYPE1,TYPE2,ABILITY1,ABILITY2,ABILITY3,TOTAL
1,1,Bulbasaur,45,49,49,65,65,45,grass,poison,Overgrow,Chlorophyll,None,318
2,2,Ivysaur,60,62,63,80,80,60,grass,poison,Overgrow,Chlorophyll,None,405
3,3,Venusaur,80,82,83,100,100,80,grass,poison,Overgrow,Chlorophyll,None,525
4,4,Charmander,39,52,43,60,50,65,fire,None,Blaze,Solar Power,None,309
5,5,Charmeleon,58,64,58,80,65,80,fire,None,Blaze,Solar Power,None,405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,901,Ursaluna,130,140,105,45,80,50,ground,normal,Guts,Bulletproof,Unnerve,550
902,902,Basculegion,120,112,65,80,75,78,water,ghost,Rattled,Adaptability,Mold Breaker,530
903,903,Sneasler,80,130,60,40,80,120,fighting,poison,Pressure,Poison Touch,None,510
904,904,Overqwil,85,115,95,65,65,85,dark,poison,Poison Point,Swift Swim,Intimidate,510


In [16]:
generation = []
gen1 = 151
gen2 = 251
gen3 = 386
gen4 = 493
gen5 = 649
gen6 = 721
gen7 = 809
gen8 = 905
i = 1

while i < 906:
    if i <= gen1:
        generation.append(1)
    elif i <= gen2:
        generation.append(2)
    elif i <= gen3:
        generation.append(3)
    elif i <= gen4:
        generation.append(4)
    elif i <= gen5:
        generation.append(5)
    elif i <= gen6:
        generation.append(6)
    elif i <= gen7:
        generation.append(7)
    else:
        generation.append(8)
    i = i+1
df['GENERATION'] = generation

In [17]:
with open ('uptogen8pokemon.pickle', 'wb') as f:
        pickle.dump(df,f)